In [1]:
import gym

In [2]:
env = gym.make("SpaceInvaders-v0", render_mode="human")

C:\Users\Sai Bhargav\AppData\Roaming\Python\Python310\site-packages\gym\envs\registration.py:505: UserWarning: WARN: The environment SpaceInvaders-v0 is out of date. You should consider upgrading to version `v5` with the environment ID `ALE/SpaceInvaders-v5`.
  logger.warn(


make the agent take random actions and see how much it gets as return.


In [3]:
episodes = 5

for episode in range(1,episodes):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        #let us make our agent take random actions initially
        state, reward, done, info = env.step(env.action_space.sample())
        #This has 6 random acitons we can take.
        #Increment the reward everytime
        score +=  reward
    print('Episode: {}\n Score :{}'.format(episode, score))


env.close()

Episode: 1
 Score :135.0
Episode: 2
 Score :105.0
Episode: 3
 Score :110.0
Episode: 4
 Score :225.0


We now would like to implement a neural network on our model to read what is going on in the scene.

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam


In [4]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [5]:
env.action_space.n

6

build a neural network to get a perception of the agent.

In [7]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32,(8,8),strides=(4,4),activation='relu',input_shape=(3, height, width, channels)))
    model.add(Conv2D(64,(4,4),strides=(2,2),activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

    

In [8]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

delete the currently built model to use the neural network.

In [7]:
del model

In [9]:
model = build_model(height, width, channels,actions)

now our model understands what is going on, so we add the reinforce layer now.

In [10]:
#form keras rl2 
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy



we use a DQN agent to use a actor-critic based Reinforcement algorithm. We use Epsilon greedy policy for the exploration part.

In [11]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=0.1, value_test=0.2, nb_steps=10000)
    memory = SequentialMemory(limit=2000, window_length = 3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, enable_dueling_network=True, dueling_type='avg', nb_actions = actions, nb_steps_warmup = 1000)

    return dqn


In [ ]:
dqn = build_agent(model, actions)

we finally build this model and visualise it as we want.

In [ ]:
dqn.compile(Adam(lr=0.0001))
dqn.fit(env,nb_steps = 10000, visualize=True, verbose = 1)

now test the model

In [ ]:
scores = dqn.test(env,nb_episodes = 10, visualize=True)
print(np.mean(scores.history['episode_reward']))

In [ ]:
dqn.save_weights('models/dqn.h5f')
dqn.load_weights('models/dqn.h5f')